FINAL PROJECT
Dora John and Veda Kamaraju

In [1]:
import pandas as pd
import numpy as np
from pdb import set_trace as st
from copy import copy
import seaborn as sns
import fastcluster
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from scipy import stats

from matplotlib.patches import Patch
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from statsmodels.stats.multitest import multipletests

In [2]:
expression_matrix = 'ExpressionData.csv'

mRNA_top = pd.read_csv(expression_matrix, delimiter='\t', skiprows=[0,1], nrows=5)

# Annotations File

Annotations_File = 'AnnotationsFile.csv'
annotations = pd.read_csv(Annotations_File, delimiter=',')



In [3]:
region_col = 'Sample Characteristic[organism part]'
regions = annotations[region_col]


import seaborn as sns

region_col = 'Sample Characteristic[organism part]'
unique_regions = sorted(annotations[region_col].unique())
print(unique_regions)


['entorhinal cortex', 'hippocampus', 'middle temporal gyrus', 'posterior cingulate cortex', 'primary visual cortex', 'superior frontal gyrus']


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib import colors as mcolors
from copy import copy

expr = pd.read_csv('ExpressionData.csv', delimiter=',')
expr = expr.rename(columns={expr.columns[0]: 'Gene_ID'})

expr.set_index('Gene_ID', inplace=True)

data = copy(expr.iloc[:, 2:])

# Drop rows with NaN or 0 values
data = data.dropna()
data = data[(data != 0).all(axis=1)]

data.columns = data.columns.str.strip()


data_row_norm = data.div(data.mean(axis=1), axis=0)
data_col_norm = data_row_norm.div(data_row_norm.mean(axis=1), axis=0)

# Log2 transform
data_log2 = np.log2(data_col_norm)
print(data_log2.columns)


metadata = pd.DataFrame({
    'Sample': data_log2.columns,
    'Region': annotations['Sample Characteristic[organism part]'].values,
    'Disease': annotations['Sample Characteristic[disease]'].values,
    'Gender': annotations['Sample Characteristic[sex]'].values,
})

metadata['Individual'] = np.arange(1, len(metadata)+1)


metadata.set_index('Sample', inplace=True)

top_genes = data_log2.var(axis=1).sort_values(ascending=False).head(200).index
data_top = data_log2.loc[top_genes]


Index(['GSM119653', 'GSM119629', 'GSM238802', 'GSM238953', 'GSM119687',
       'GSM119647', 'GSM119643', 'GSM119660', 'GSM238837', 'GSM238942',
       ...
       'GSM119662', 'GSM119635', 'GSM119625', 'GSM119651', 'GSM119646',
       'GSM238870', 'GSM238839', 'GSM119688', 'GSM238815', 'GSM119683'],
      dtype='object', length=161)


In this portion of the code, the dataset is loaded for gene expression and sample metadata. Initially, the code filters and normalizes the data. Following, top variable genes are selected and the data is prepared for binary classification of those samples which are Alzheimer's or control, pulling their respective gene expressions. A neural network is then trained to predict disease status from expression and test accuracy is reported. 

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import pandas as pd

X = data_top.T.values  
y_labels = metadata.loc[data_top.columns, 'Disease'].values 

le = LabelEncoder()
y = le.fit_transform(y_labels)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# SPLIT DATA
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

class FullyConnectedNN1(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = FullyConnectedNN1(X_train, y_train)
test_dataset = FullyConnectedNN1(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

class FullyConnectedNN1(nn.Module):
    def __init__(self, input_dim):
        super(FullyConnectedNN1, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.sigmoid(self.fc3(x))
        return x

model = FullyConnectedNN1(input_dim=X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 30
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predicted = (outputs > 0.5).float()
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Epoch 1/30, Loss: 0.0000
Epoch 2/30, Loss: 0.0000
Epoch 3/30, Loss: 0.0000
Epoch 4/30, Loss: 0.0000
Epoch 5/30, Loss: 0.0000
Epoch 6/30, Loss: 0.0000
Epoch 7/30, Loss: 0.0000
Epoch 8/30, Loss: 0.0000
Epoch 9/30, Loss: 0.0000
Epoch 10/30, Loss: 0.0000
Epoch 11/30, Loss: 0.0000
Epoch 12/30, Loss: 0.0000
Epoch 13/30, Loss: 0.0000
Epoch 14/30, Loss: 0.0000
Epoch 15/30, Loss: 0.0000
Epoch 16/30, Loss: 0.0000
Epoch 17/30, Loss: 0.0000
Epoch 18/30, Loss: 0.0000
Epoch 19/30, Loss: 0.0000
Epoch 20/30, Loss: 0.0000
Epoch 21/30, Loss: 0.0000
Epoch 22/30, Loss: 0.0000
Epoch 23/30, Loss: 0.0000
Epoch 24/30, Loss: 0.0000
Epoch 25/30, Loss: 0.0000
Epoch 26/30, Loss: 0.0000
Epoch 27/30, Loss: 0.0000
Epoch 28/30, Loss: 0.0000
Epoch 29/30, Loss: 0.0000
Epoch 30/30, Loss: 0.0000
Test Accuracy: 57.58%


In [8]:
# TCA CYCLE

import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# HAVE TO LOAD IT AGAIN AND CLEAN IT UP TO MAP TO TCA FILE

expression_data = pd.read_csv("ExpressionData.csv")
expression_data_clean = expression_data.drop(columns=['Gene ID', 'DesignElementAccession'])
expression_data_clean.set_index('Gene Name', inplace=True)
expression_data_clean = expression_data_clean.apply(pd.to_numeric, errors='coerce')
data_top = np.log2(expression_data_clean + 1)

def GetPathway(FilePath):
    with open(FilePath, 'r') as file:
        matches = re.findall(r'\b([A-Z0-9]+);', file.read())
    return np.array(matches)

Pathway_Genes = np.char.strip(GetPathway("tca.txt"))
Pathway_Data = data_top.loc[data_top.index.intersection(Pathway_Genes)]

metadata = pd.read_csv("AnnotationsFile.csv")
metadata.set_index("Assay", inplace=True)
sample_labels = metadata.loc[Pathway_Data.columns, 'Sample Characteristic[disease]'].values

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(sample_labels)
label_to_int = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
int_to_label = {v: k for k, v in label_to_int.items()}

train_cols, val_cols, train_labels, val_labels = train_test_split(
    Pathway_Data.columns,
    encoded_labels,
    test_size=0.3,
    stratify=encoded_labels,
    random_state=42
)

class TranscriptomicsDataset(Dataset):
    def __init__(self, data, columns, labels):
        self.data = data[columns].T
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = torch.tensor(self.data.iloc[idx].values, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sample, label

train_dataset = TranscriptomicsDataset(Pathway_Data, train_cols, train_labels)
val_dataset = TranscriptomicsDataset(Pathway_Data, val_cols, val_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

class FullyConnectedNN2(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(FullyConnectedNN2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

input_size = len(Pathway_Data.index)
model = FullyConnectedNN2(input_size, 16, 8, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def evaluate(loader):
    total_loss, correct, total = 0.0, 0, 0
    model.eval()
    with torch.no_grad():
        for data, labels in loader:
            outputs = model(data)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / len(loader), correct / total

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss, correct_train, total_train = 0.0, 0, 0
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    
    train_loss = running_loss / len(train_loader)
    train_acc = correct_train / total_train
    val_loss, val_acc = evaluate(val_loader)

print("Training Accruacy: ", train_acc)
print("Validation Accuracy: ", val_acc)

torch.save(model.state_dict(), 'trained_model.pth')

trained_model = FullyConnectedNN2(input_size, 16, 8, 2)
trained_model.load_state_dict(torch.load("trained_model.pth"))
trained_model.eval()

print("Matched genes:", list(Pathway_Data.index))
print("Number of matched genes:", len(Pathway_Data))

# TEST EXAMPLE 

sample_id = 'GSM119687'
Test_Array = torch.from_numpy(Pathway_Data.loc[:, sample_id].to_numpy()).float().unsqueeze(0)
output = trained_model(Test_Array)
_, predicted_class = torch.max(output, 1)
predicted_class_name = int_to_label[predicted_class.item()]
print("Predicted class for", sample_id, ":", predicted_class_name)


Training Accruacy:  0.7589285714285714
Validation Accuracy:  0.7755102040816326
Matched genes: ['MDH1', 'IDH3G', 'DLD', 'FH', 'ACO2', 'PCK2', 'IDH3B', 'OGDH', 'SDHB', 'DLST', 'ACO1', 'PCK1', 'ACLY', 'PDHA1', 'SUCLA2', 'IDH1', 'MDH2', 'DLAT', 'PDHA2', 'SUCLG1', 'IDH3A', 'PDHB', 'SUCLG2', 'PC', 'IDH2', 'OGDHL', 'SDHD']
Number of matched genes: 27
Predicted class for GSM119687 : Alzheimers disease


In this portion of the code, initially there are a sequence of imports, metadata is loaded about the samples, and organ/tissue region is pulled out. The entire expression matrix is read and gene identifiers are set as the DataFrame index. The data is log-transformed and a sample metadata table is constructed. The top 200 most variable genes across all samples are selected for analysis. To prepare the data for classification, the gene matrix is transposed to a shape and disease status is encoded as 0 or 1; the data is split into testing/training sets. The PyTorch dataset is then used, which wraps the dataset into loaders. The fully connected neural network is then established, with a training loop for a loss function (utilizing the Adam optimizer), and training is run for 30 epochs (includes forward pass, loss computation, backpropagation, and updated weights). Lastly, the model was evaluated on a test set which revealed a 76% testing accuracy and 78% validation accuracy. 

Our training model ultimately classified tissue as Alzheimers disease or control based on the expression of TCA cycle genes which are implicated in energy metabolism. The model correctly classified 76% of the training samples and learned the pattern in the training data. It also predicted 78% of the unseen samples -- validation accuracy being slightly higher in the training shows that there is no overfitting and the model generalizes well. There are about 27 genes indicated as being matched to an unknown gene list. In attempting to match model-selected genes to known biology like altered energy metabolis, this is predictive of strong biological interpretability. IDH3A, PDHA1, and OGDH are all mitochondrial/TCA-cycel related, whose alteration in expression makes sense to be downregulated in Alzheimer's disease (decreased metabolism). 